In [2]:
from flask import Flask,jsonify,request
import json
import pandas as pd
import numpy as np
import os

# from sklearn.cluster import KMeans
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #limit floats output to 3
pd.set_option('display.max_columns', None) #display all columns in data frame
pd.set_option('display.max_rows', None)
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# %matplotlib inline
# import googlemaps
from datetime import datetime
import math
from k_means_constrained import KMeansConstrained
import pyodbc 

In [5]:
rawdatatable = pd.read_csv(r'C:\IIIfinalproject\final project\python\Berkeley_1000.csv',encoding="ISO-8859-1")

In [11]:
# rawdatatable1 = pd.read_csv(r'./Berkeley_1000.csv',encoding="ISO-8859-1")
rawdatatable1 = pd.read_csv(r'./Berkeley_1000.csv')

In [16]:
rawdatatable.columns
rawdatatable.head()

,Latitude,Longitude,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,-122.263,37.884,1322,EUCLID AV,2,BERKELEY,NaN,NaN,94708.000,59-2249-7,49d10233a61c99d4
1,-122.272,37.890,2001,LOS ANGELES AVE,NaN,BERKELEY,NaN,NaN,94707.000,061 258602000,8d385385e62d7858
2,-122.267,37.872,2119,ADDISON ST,24,BERKELEY,NaN,NaN,94704.000,57-2034-5,293090578dc1c7e3
3,-122.269,37.874,2030,DELAWARE ST,NaN,BERKELEY,NaN,NaN,94709.000,57-2050-29-1,ece852a0536f2dd7
4,-122.266,37.888,1132,ARCH ST,NaN,BERKELEY,NaN,NaN,94708.000,061 255600500,143f6e3f4d200cae


In [31]:
rawdatatable = pd.read_csv(r'C:\IIIfinalproject\final project\python\Berkeley_1000.csv',header=None)
rawdatatable.columns = ['Latitude','Longitude','NUMBER','STREET','UNIT','CITY','DISTRICT','REGION','POSTCODE','ID','HASH']


rawdatatable["ADDRESS"] = rawdatatable["STREET"] + rawdatatable["UNIT"]

# Number of clusters (Number of working days)
Target_Num_Cluster = 22
avg_sites = round(rawdatatable1.shape[0]/22,0)
print('Expected number of clusters:',Target_Num_Cluster)
X_features = rawdatatable[['Longitude','Latitude']]

# Size constraint K-means
clf = KMeansConstrained(
n_clusters=Target_Num_Cluster,
size_min=avg_sites-3,
size_max=avg_sites+3,
random_state=0
)
clf.fit_predict(X_features)


# Get the labels of each store
Label_frame = pd.DataFrame(clf.labels_)
# Get the number of store which allocate to certain clusters
for i in range(Target_Num_Cluster):
    print('Cluster',i,":",clf.labels_[clf.labels_==i].shape[0],'stores')

Storeinfo = rawdatatable[['NUMBER','POSTCODE','ADDRESS']]
clustering_X_features = pd.concat([Storeinfo,X_features,Label_frame], axis=1)
clustering_X_features = clustering_X_features.rename(columns={0 : 'Clusters'})
clustering_X_features.head()

dataset=[]
for i in range(Target_Num_Cluster):
    dataset.append({
    'Clusters':i,
    'datas':[]
    })
    clusters = clustering_X_features[clustering_X_features['Clusters']==i]
    cluster_len = len(clusters)

    for j in range(cluster_len):
        LatLngObj = {'lat':clusters.iloc[j,:][3],'lng':clusters.iloc[j,:][4]}
        dataset[i]['datas'].append({
            'NUMBER':str(clusters.iloc[j,:][0]),
            'POSTCODE':str(clusters.iloc[j,:][1]),
            'ADDRESS':str(clusters.iloc[j,:][2]),
            'Longitude':str(clusters.iloc[j,:][3]),
            'Latitude':str(clusters.iloc[j,:][4]),
            'LatLng':str(LatLngObj)
        })
print('Clustering Success')

Expected number of clusters: 22
Cluster 0 : 44 stores
Cluster 1 : 42 stores
Cluster 2 : 48 stores
Cluster 3 : 42 stores
Cluster 4 : 46 stores
Cluster 5 : 45 stores
Cluster 6 : 48 stores
Cluster 7 : 48 stores
Cluster 8 : 48 stores
Cluster 9 : 42 stores
Cluster 10 : 45 stores
Cluster 11 : 48 stores
Cluster 12 : 42 stores
Cluster 13 : 43 stores
Cluster 14 : 43 stores
Cluster 15 : 47 stores
Cluster 16 : 42 stores
Cluster 17 : 48 stores
Cluster 18 : 45 stores
Cluster 19 : 48 stores
Cluster 20 : 48 stores
Cluster 21 : 48 stores
Clusering Success


In [38]:
rawdatatable['Longitude']

0     37.884
1     37.890
2     37.872
3     37.874
4     37.888
5     37.863
6     37.862
7     37.866
8     37.900
9     37.889
10    37.852
11    37.857
12    37.883
13    37.879
14    37.865
15    37.856
16    37.880
17    37.853
18    37.863
19    37.876
20    37.868
21    37.865
22    37.866
23    37.878
24    37.854
25    37.865
26    37.857
27    37.874
28    37.871
29    37.867
30    37.879
31    37.853
32    37.882
33    37.879
34    37.897
35    37.856
36    37.867
37    37.867
38    37.890
39    37.865
40    37.897
41    37.884
42    37.865
43    37.890
44    37.869
45    37.866
46    37.869
47    37.878
48    37.871
49    37.860
50    37.864
51    37.871
52    37.864
53    37.868
54    37.861
55    37.876
56    37.873
57    37.901
58    37.880
59    37.875
60    37.902
61    37.889
62    37.896
63    37.864
64    37.900
65    37.855
66    37.867
67    37.880
68    37.879
69    37.857
70    37.877
71    37.877
72    37.895
73    37.883
74    37.882
75    37.876
76    37.891